In [1]:
from api.chains.off_topic_chain import create_off_topic_chain
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name = "gpt-3.5-turbo", streaming = True)

/Users/danliu/miniconda3/envs/chatbot/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
from api.chains.qa_chain_with_rag import create_llm_finetun_chain
llm_fine_tune_chain = create_llm_finetun_chain(llm)
llm_fine_tune_chain.invoke({"question": "what is finetuning"})

"Fine-tuning is the process of taking a pre-trained model and further training it on a specific dataset or task to improve its performance on that particular task. In the context of AutoTrain documentation from Hugging Face, fine-tuning large language models (LLMs) involves adjusting the model's parameters based on new data to enhance its performance on tasks like causal language modeling (CLM) and masked language modeling (MLM)."

In [7]:
from api.chains.classification_chain import create_classification_chain
classification_chain = create_classification_chain(llm)
classification_chain.invoke({"question": "color of sky"})

'Other'

In [2]:
from api.full_chain import init_full_chain
full_chain = init_full_chain(llm)
full_chain.invoke({"question": "what is the llm finetuning", "memory": "eee"})

'LLM finetuning refers to the process of fine-tuning large language models (LLMs) on your own data using AutoTrain. It supports different types of LLM finetuning, such as Causal Language Modeling (CLM) and Masked Language Modeling (MLM). The data for LLM finetuning should be in CSV format, and there are specific data formats required for different types of trainers within AutoTrain, such as SFT/Generic Trainer, Reward Trainer, and DPO Trainer. The parameters for running AutoTrain LLM include options for training, deploying, and running inference, as well as specifying the model, project name, data path, batch size, epochs, learning rate, optimizer, scheduler, and other training configurations.'